<a href="https://colab.research.google.com/github/mavenceslau/Challenge-Data-Science-Alura-Cash/blob/master/notebooks/Notebook_03_Modelos_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 02 - Aprendendo com os dados: criando um modelo de previsão de inadimplência - Modelos de Machine Learning

Chegou a hora de criar os modelos de machine learning para classificação, além de usar métricas para testar a eficácia dos mesmos.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from utils import config_exibicao, config_graficos, box, remove_outlier

Com a função **config_exibicao** iremos configurar a exibição de dataframes pandas, de modo que os mesmos possam exibir o maior número de informações nas células sem cortar conteudo.

E com a função **config_graficos** vamos configurar a exibição dos gráficos.

Mais informações sobre essas funções podem ser vistas no arquivo [utils.py](https://github.com/mavenceslau/Challenge-Data-Science-Alura-Cash/blob/master/notebooks/utils.py)

In [ ]:
config_exibicao()
config_graficos()

Importando a base de dados e exibindo os primeiros registros

In [ ]:
url = '../data/processed/alura_cash_tratado.csv'
dados = pd.read_csv(url)
dados.head()

,pessoa_idade,pessoa_salario_anual,pessoa_status_propriedade,pessoa_tempo_trabalho,emprestimo_motivo,emprestimo_pontuacao,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_inadimplencia,historico_tempo_credito
0,22,50000.0,Hipotecada,0,Pagamento de débitos,A,12800.0,5.79,0,0.26,N,2.0
1,26,84996.0,Alugada,0,Médico,B,9000.0,11.26,0,0.11,N,2.0
2,24,55782.0,Alugada,3,Melhora do lar,A,17000.0,7.51,0,0.30,N,2.0
3,25,125000.0,Própria,9,Pessoal,B,25000.0,10.59,0,0.20,N,2.0
4,23,44340.0,Própria,0,Empreendimento,C,3000.0,12.98,0,0.07,S,3.0


In [ ]:
print(f'O dataset possui {dados.shape[0]} linhas e {dados.shape[1]} colunas')

O dataset possui 32316 linhas e 12 colunas


## Aplicando Encoding nas váriaveis

In [ ]:
categorias = ['pessoa_status_propriedade', 'emprestimo_motivo', 'emprestimo_pontuacao', 'historico_inadimplencia']
dados2 = dados.drop(categorias, axis=1)
dados2.columns

Index(['pessoa_idade', 'pessoa_salario_anual', 'pessoa_tempo_trabalho',
       'emprestimo_valor_total', 'emprestimo_taxa_juros',
       'emprestimo_inadimplente', 'emprestimo_renda_percentual',
       'historico_tempo_credito'],
      dtype='object')

In [ ]:
ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(dados[categorias]).toarray()
dados_final = pd.concat([dados2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(categorias))], axis=1)
dados_final.head()

,pessoa_idade,pessoa_salario_anual,pessoa_tempo_trabalho,emprestimo_valor_total,emprestimo_taxa_juros,emprestimo_inadimplente,emprestimo_renda_percentual,historico_tempo_credito,pessoa_status_propriedade_Alugada,pessoa_status_propriedade_Hipotecada,pessoa_status_propriedade_Outros,pessoa_status_propriedade_Própria,emprestimo_motivo_Educativo,emprestimo_motivo_Empreendimento,emprestimo_motivo_Melhora do lar,emprestimo_motivo_Médico,emprestimo_motivo_Pagamento de débitos,emprestimo_motivo_Pessoal,emprestimo_pontuacao_A,emprestimo_pontuacao_B,emprestimo_pontuacao_C,emprestimo_pontuacao_D,emprestimo_pontuacao_E,emprestimo_pontuacao_F,emprestimo_pontuacao_G,historico_inadimplencia_N,historico_inadimplencia_S
0,22,50000.0,0,12800.0,5.79,0,0.26,2.0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
1,26,84996.0,0,9000.0,11.26,0,0.11,2.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
2,24,55782.0,3,17000.0,7.51,0,0.30,2.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
3,25,125000.0,9,25000.0,10.59,0,0.20,2.0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
4,23,44340.0,0,3000.0,12.98,0,0.07,3.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
